# OPER 623 - Heuristist Search Methods
## Homework 1
### Hosley, Brandon

In [1]:
from ortools.linear_solver import pywraplp

*We are going to transform this problem into one that is slightly more complicated called the 0-1 multi-constraint multi-knapsack problem.  The difference between these two problems is that the 0-1 multi-knapsack problem has only one constraint, weight, that you must control. Whereas the multi-constraint multi-knapsack problem has multiple constraints.  In both cases the goal is to maximize value, while ensuring you meet constraints.*

We will use the following sets and variables </br>

$B$ : the set of knapsacks </br>
$I$ : the set of items

$x_{i,b}$ : a truth value corresponding to if item $i$ is packed within knapsack $b$ </br>
$y_i$ : the value of item $i$ </br>
$w_i$ : the weight of item $i$ </br>
$v_i$ : the volume of item $i$ </br>
$r_i$ : the radioactivity of item $i$ </br>

to define the mathematical model as

$$\begin{align*}
\max \sum_{i\in I, b\in B} x_{i,b}\, y_i & \\
\text{s.t.} \quad
\sum_{i\in I}x_{i,b}\, w_i &\leq 50 \quad\forall\,b\in B  
\qquad\text{(weight constraint)}\\
\sum_{i\in I}x_{i,b}\, v_i &\leq 50 \quad\forall\,b\in B  \qquad\text{(volume constraint)}\\
\sum_{i\in I}x_{i,b}\, r_i &\leq 5\,\ \quad\forall\,b\in B 
\qquad\text{(radioactivity constraint)} \\
x_{i,b} &\in \{0,1\} \quad\forall\,i\in I,\,b\in B \\
\end{align*}$$

To further condense this model we will collect the constraints into a single structure. </br>

Let the constraint values for each item $i$ be $d_{c,i}$ where $c \in \{\text{weight, volume, radioactivity}\}$ </br>
and the capacity for each constraint $c$ in each bin is $k_c$.

Giving,

$$\begin{align*}
\max \sum_{i\in I, b\in B} x_{i,b}\, y_i & \\
\text{s.t.} \quad
\sum_{i\in I}x_{i,b}\, d_{c,i} &\leq k_c \,\,\qquad\forall\,b\in B,\forall\,c\in C \\
x_{i,b} &\in \{0,1\} \quad\forall\,i\in I,\,b\in B \\
\end{align*}$$

First, we transcribe the data into Python data structures.

In [2]:
data = {
    "value":           [48,	30,	42,	36,	22,	43,	18,	24,	36,	29,	30,	25,	19,	41,	34,	32,	27,	24,	18], 
    "weight":         [10,	30,	12,	22,	12,	20,	9,	9,	18,	20,	25,	18,	7,	16,	24,	21,	21,	32,	9 ], 
    "volume":          [15,	20,	18,	20,	5,	12,	7,	7,	24,	30,	25,	20,	5,	25,	19,	24,	19,	14,	30], 
    "radioactivity":   [3,	1,	2,	3,	1,	2,	0,	2,	2,	1,	2,	3,	4,	3,	2,	3,	1,	1,	3 ]
}

constraints = {
    "weight": 50, "volume": 50, "radioactivity": 5
}

num_bins = 5
bins = list(range(num_bins))
items = range(len(data["value"]))

Here we instantiate a solver. We opt to use SCIP rather than GLOP because it seems that GLOP is only able to handle LPs. SCIP however, can handle IPs; which is what we have until the 'items' can be divided into smaller parts.

In [3]:
solver = pywraplp.Solver.CreateSolver('SCIP')

Next, we create a matrix holding boolean values correlating item to knapsack. To save resources we also add the constraint that an item can only be contained in a single knapsack.
$$ x_{i,b} \in \{0,1\} \quad\forall\,i\in I,\,b\in B $$

In [4]:
x = {}
for i in items:
    for b in bins:
        x[i, b] = solver.BoolVar(f"x_{i}_{b}")
    # Each item is assigned only once
    solver.Add(sum(x[i, b] for b in bins) <= 1)

Then we add the knapsack capacity and and person's radioactivity constraints.
$$\sum_{i\in I}x_{i,b}\, d_{c,i} \leq k_c \qquad\forall\,b\in B,\forall\,c\in C \\$$

In [5]:
for c in constraints:
    for b in bins:
        solver.Add(
            sum(x[i, b] * data[c][i] for i in items) <= constraints[c] 
        )

Next, we translate the objective function
$$\max \sum_{i\in I, b\in B} x_{i,b}\, y_i.$$

In [6]:
# Maximize total value of packed items.
objective = solver.Objective()
for i in items:
    for b in bins:
        objective.SetCoefficient(x[i, b], data["value"][i])
objective.SetMaximization()

Having define the problem we can call the solver to solve.

In [7]:
status = solver.Solve()

Finally, we display the results. 
*The method of doing so is largely taken from the Google OR-Tools [example](https://developers.google.com/optimization/pack/multiple_knapsack).*

In [9]:
total_value, total_wgt, total_vol, total_rad = 0,0,0,0
for b in bins:
    print(f"Knapsack {b}:")
    bin_value, bin_wgt, bin_vol, bin_rad = 0,0,0,0

    for i in items:
        if x[i, b].solution_value() > 0:
            y = data['value'][i]
            w = data['weight'][i]
            v = data['volume'][i]
            r = data['radioactivity'][i]
            print(f"Item {i} - value: {y}, weight: {w}, volume: {v}, radioactivity: {r}")
            bin_value, bin_wgt, bin_vol, bin_rad = bin_value+y, bin_wgt+w, bin_vol+v, bin_rad+r

    print(f"Sack Weight: {bin_wgt} \nSack Volume: {bin_vol} \nSack Radioactivity: {bin_rad}\n")

    total_value, total_wgt, total_vol, total_rad = total_value+bin_value, total_wgt+bin_wgt, total_vol+bin_vol, total_rad+bin_rad

print(f"\nTotal Value: {total_value} \nTotal Weight: {total_wgt} \
      \nTotal Volume: {total_vol} \nTotal Radioactivity: {total_rad}\n")

Knapsack 0:
Item 0 - value: 48, weight: 10, volume: 15, radioactivity: 3
Item 6 - value: 18, weight: 9, volume: 7, radioactivity: 0
Item 14 - value: 34, weight: 24, volume: 19, radioactivity: 2
Sack Weight: 43 
Sack Volume: 41 
Sack Radioactivity: 5

Knapsack 1:
Item 1 - value: 30, weight: 30, volume: 20, radioactivity: 1
Item 13 - value: 41, weight: 16, volume: 25, radioactivity: 3
Sack Weight: 46 
Sack Volume: 45 
Sack Radioactivity: 4

Knapsack 2:
Item 2 - value: 42, weight: 12, volume: 18, radioactivity: 2
Item 4 - value: 22, weight: 12, volume: 5, radioactivity: 1
Item 10 - value: 30, weight: 25, volume: 25, radioactivity: 2
Sack Weight: 49 
Sack Volume: 48 
Sack Radioactivity: 5

Knapsack 3:
Item 3 - value: 36, weight: 22, volume: 20, radioactivity: 3
Item 8 - value: 36, weight: 18, volume: 24, radioactivity: 2
Sack Weight: 40 
Sack Volume: 44 
Sack Radioactivity: 5

Knapsack 4:
Item 5 - value: 43, weight: 20, volume: 12, radioactivity: 2
Item 7 - value: 24, weight: 9, volume: 7,